In [1]:
import pandas as pd
import numpy as np
import json
import os
import datetime as dt
import requests
import sys
#from tqdm.auto import tqdm
#import math

In [2]:
zillow_home_index_file = "data/Zip_Zhvi_Summary_AllHomes.csv"
zip_code_file = "data/zip_code_list.txt"
zip_code_stats = "data/Zip_Code_Raw_Data.csv"

## Realtor.com info.
This is the historical data file from the realtor.com residential listings database, 
used with permission from https://www.realtor.com/research/data

Note: the realtor_data_file (RDC_InventoryCoreMetrics_Zip_Hist.csv) is too large to be stored in Github.   
The following cell will check for the existence of the file, and download if not present. 
It will take maybe five minutes, depending on your connection.

In [3]:
realtor_data_source = "https://s3-us-west-2.amazonaws.com/econresearch/Reports/Core/RDC_InventoryCoreMetrics_Zip_Hist.csv"
realtor_data_file = os.path.join("data", "RDC_InventoryCoreMetrics_Zip_Hist.csv")

# Test for local file. If exists, read it in. If not, download it. 
# We should just check the etag for file changes and re-download
# it anyway if newer file exists.

# Test if file exists and is not empty.
# If not, download the file.
# Then read into Pandas.

exists = os.path.isfile(realtor_data_file)
if exists:
        bytesize = os.path.getsize(realtor_data_file)
    
if not exists or bytesize <= 0:
    #print(bytesize)
    print("Realtor.com file not found or is empty. Downloading now.")
    print("This could take ~5 min or so, depending on your connection.\n")
    print("Progress:")
    with requests.get(realtor_data_source, stream=True) as r:
        r.raise_for_status()
        total_size = int(r.headers.get('content-length', 0))
        
        with open(realtor_data_file, 'wb') as f:
            dl = 0
            for chunk in r.iter_content(chunk_size=4096): 
                if chunk:  # filter out keep-alive new chunks
                    f.write(chunk)
                    dl = dl + len(chunk)
                    done = int(50*dl / total_size)
                    sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )    
                    sys.stdout.flush()
            
    print("\nFile downloaded.")
    exists = os.path.isfile(realtor_data_file)
    if exists:
        bytesize = os.path.getsize(realtor_data_file)
    if not exists or bytesize <=0:
        print("Looks like the download is b0rked.")
        print("Try getting the file manually.")

print("Reading File...\n")
realtor_df = pd.read_csv(realtor_data_file, engine='python', encoding="ISO-8859-1", dtype={'Month':'str', 'ZipCode':'str', 'Footnote':'str'}, skipfooter=1)
print("Done")

Realtor.com file not found or is empty. Downloading now.
This could take ~5 min or so, depending on your connection.

Progress:
[==================================================]
File downloaded.
Reading File...

Done


### Wait until progress bar above has completed!

In [4]:
#realtor_df.info()

In [5]:
# Convert month column to datetime object.
realtor_df['Month'] = pd.to_datetime(realtor_df['Month'])

# dtype should be <M8[ns] on little-endian machines (Intel), >M8[ns] on big-endian machines (Sparc, PPC, etc)
#realtor_df['Month'].dtype

In [6]:
realtor_df.head()

,Month,ZipCode,ZipName,Footnote,Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Active Listing Count,Active Listing Count M/M,Active Listing Count Y/Y,...,Pending Listing Count Y/Y,Avg Listing Price,Avg Listing Price M/M,Avg Listing Price Y/Y,Total Listing Count,Total Listing Count M/M,Total Listing Count Y/Y,Pending Ratio,Pending Ratio M/M,Pending Ratio Y/Y
0,2019-02-01,01001,"Agawam, MA",*,209500.0,0.0411,NaN,47.0,-0.1532,-0.1132,...,0.0000,242888.0,0.0217,-0.1314,48.0,-0.1504,-0.1111,0.0213,0.0033,0.0024
1,2019-02-01,01002,"Amherst, MA",nan,447225.0,-0.0150,0.1540,56.0,0.1200,-0.2483,...,NaN,493027.0,-0.0132,0.0617,56.0,0.1200,-0.2483,0.0000,0.0000,0.0000
2,2019-02-01,01005,"Barre, MA",*,241550.0,-0.0069,0.0679,16.5,-0.3529,0.0313,...,NaN,343905.0,0.0343,0.6497,16.5,-0.3529,0.0313,0.0000,0.0000,0.0000
3,2019-02-01,01007,"Belchertown, MA",nan,331225.0,-0.0076,-0.0398,49.0,0.0889,-0.2033,...,NaN,324858.0,-0.0221,-0.0564,49.0,0.0889,-0.2033,0.0000,0.0000,0.0000
4,2019-02-01,01008,"Blandford, MA",*,216275.0,-0.0333,NaN,9.5,0.0556,NaN,...,0.3333,224871.0,-0.0243,0.0409,11.5,0.0455,0.4375,0.2105,-0.0117,-0.0202


In [7]:
filtered_realtor_df = realtor_df[realtor_df['Month'].dt.year == 2018]

In [8]:
filtered_realtor_df.head(5)

,Month,ZipCode,ZipName,Footnote,Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Active Listing Count,Active Listing Count M/M,Active Listing Count Y/Y,...,Pending Listing Count Y/Y,Avg Listing Price,Avg Listing Price M/M,Avg Listing Price Y/Y,Total Listing Count,Total Listing Count M/M,Total Listing Count Y/Y,Pending Ratio,Pending Ratio M/M,Pending Ratio Y/Y
30057,2018-12-01,01001,"Agawam, MA",nan,208300.0,0.0158,-0.0309,64.0,-0.1233,0.2308,...,0.0,247625.0,-0.0032,0.0154,65.0,-0.1216,0.2264,0.0156,0.0019,-0.0036
30058,2018-12-01,01002,"Amherst, MA",nan,425050.0,-0.0234,0.2503,61.0,-0.2278,-0.1974,...,NaN,480597.0,0.0229,0.0655,61.0,-0.2278,-0.1974,0.0000,0.0000,0.0000
30059,2018-12-01,01005,"Barre, MA",*,240050.0,-0.0062,NaN,31.0,-0.1143,NaN,...,NaN,324806.0,0.0349,0.4040,31.0,-0.1143,0.4762,0.0000,0.0000,0.0000
30060,2018-12-01,01007,"Belchertown, MA",nan,344950.0,0.0614,0.0012,47.0,-0.1376,-0.2985,...,NaN,341111.0,-0.0122,-0.0034,47.0,-0.1376,-0.2985,0.0000,0.0000,0.0000
30061,2018-12-01,01008,"Blandford, MA",*,239950.0,-0.0266,NaN,12.0,0.0000,NaN,...,0.0,235810.0,-0.0773,-0.0587,14.0,0.0000,0.1667,0.1667,0.0000,-0.0333


In [9]:
# Read in zip code file. This is a list of zip codes along the San Andreas Fault.
zips_df = pd.read_csv(zip_code_file, encoding="ISO-8859-1", dtype={'ZipCode':'str'})

In [10]:
#zips_df.info()

In [11]:
# Get a count of zip codes in my zip list
zips_count = zips_df['ZipCode'].count()
#zips_count

In [12]:
# Filter all realtor.com info to just our zip code list.
# We do this by merging realtor data with zip code list using an inner join. 
filtered_zips_df = pd.merge(realtor_df, zips_df, on="ZipCode")

In [13]:
#filtered_zips_df.info()

In [14]:
filtered_zips_df

,Month,ZipCode,ZipName,Footnote,Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Active Listing Count,Active Listing Count M/M,Active Listing Count Y/Y,...,Pending Listing Count Y/Y,Avg Listing Price,Avg Listing Price M/M,Avg Listing Price Y/Y,Total Listing Count,Total Listing Count M/M,Total Listing Count Y/Y,Pending Ratio,Pending Ratio M/M,Pending Ratio Y/Y
0,2019-02-01,92241,"Desert Hot Springs, CA",nan,249950.00,-0.0090,0.0611,30.5,0.1961,0.1731,...,1.3333,433049.0,-0.0476,0.0214,34.0,0.1930,0.2364,0.1148,-0.0029,0.0571
1,2019-01-01,92241,"Desert Hot Springs, CA",nan,252225.00,-0.0089,-0.0271,25.5,0.0200,-0.1774,...,5.0000,454677.0,-0.0702,0.0585,28.5,0.0962,-0.0952,0.1176,0.0776,0.1015
2,2018-12-01,92241,"Desert Hot Springs, CA",nan,254500.00,-0.0745,-0.0398,25.0,-0.1525,-0.1667,...,-0.5000,488984.0,0.0374,0.2409,26.0,-0.1613,-0.1875,0.0400,-0.0108,-0.0267
3,2018-11-01,92241,"Desert Hot Springs, CA",nan,275000.00,-0.0516,0.0221,29.5,-0.0484,0.2292,...,-0.5000,471347.0,0.0932,0.4566,31.0,-0.1268,0.1481,0.0508,-0.0943,-0.0742
4,2018-10-01,92241,"Desert Hot Springs, CA",*,289950.00,0.0939,-0.0333,31.0,-0.0606,0.3478,...,3.5000,431134.0,0.3002,0.1734,35.5,-0.0405,0.4792,0.1452,0.0239,0.1017
5,2018-09-01,92241,"Desert Hot Springs, CA",nan,265050.00,0.0415,-0.1682,33.0,0.0476,0.3750,...,3.0000,331568.0,-0.1347,-0.2100,37.0,0.0423,0.4800,0.1212,-0.0058,0.0795
6,2018-08-01,92241,"Desert Hot Springs, CA",nan,254500.00,-0.0022,-0.1490,31.5,0.0500,0.4000,...,3.0000,383146.0,-0.0950,-0.0015,35.5,0.0758,0.5106,0.1270,0.0270,0.0825
7,2018-07-01,92241,"Desert Hot Springs, CA",nan,255050.00,-0.0494,0.2623,30.0,-0.0323,0.5000,...,2.0000,423352.0,0.0520,0.1112,33.0,0.0154,0.5714,0.1000,0.0516,0.0500
8,2018-06-01,92241,"Desert Hot Springs, CA",nan,268300.00,0.0367,0.3279,31.0,0.0000,0.3778,...,2.0000,402414.0,-0.0482,0.2055,32.5,0.0317,0.4130,0.0484,0.0323,0.0262
9,2018-05-01,92241,"Desert Hot Springs, CA",nan,258800.00,0.0147,0.3134,31.0,0.0000,0.4091,...,-0.8750,422781.0,-0.0550,0.2640,31.5,-0.0156,0.2115,0.0161,-0.0161,-0.1657


In [15]:
filtered_zipcode_count = len(filtered_zips_df['ZipCode'].unique())
filtered_zipcode_count
missing_zips = zips_count - filtered_zipcode_count

In [16]:
print('Total SAF Zipcodes: ' + str(zips_count))
print('SAF Zipcodes in dataset: ' + str(filtered_zipcode_count))
print('Missing Zipcodes: ' + str(missing_zips) + "\n")
print("This is a first swag, so we'll acknowldge the " + str(missing_zips) + " missing zip codes as a gap.")

Total SAF Zipcodes: 84
SAF Zipcodes in dataset: 39
Missing Zipcodes: 45

This is a first swag, so we'll acknowldge the 45 missing zip codes as a gap.


In [17]:
zip_listings = filtered_zips_df[['Month', 'ZipCode', 'ZipName', 'Median Listing Price', 'Avg Listing Price']]

In [18]:
# Why you no rename in place?
yet_another_dataframe_because_rename_doesnt_rename_in_place = zip_listings.rename(columns = {'Month':'Date'})
#yet_another_dataframe_because_rename_doesnt_rename_in_place.head()
zip_listings = yet_another_dataframe_because_rename_doesnt_rename_in_place
zip_listings.head()

,Date,ZipCode,ZipName,Median Listing Price,Avg Listing Price
0,2019-02-01,92241,"Desert Hot Springs, CA",249950.0,433049.0
1,2019-01-01,92241,"Desert Hot Springs, CA",252225.0,454677.0
2,2018-12-01,92241,"Desert Hot Springs, CA",254500.0,488984.0
3,2018-11-01,92241,"Desert Hot Springs, CA",275000.0,471347.0
4,2018-10-01,92241,"Desert Hot Springs, CA",289950.0,431134.0


In [19]:
print("There are " + str(len(zip_listings)) + " rows.")

There are 3526 rows.


## Zillow Information

In [20]:
zillow_df = pd.read_csv(zillow_home_index_file, encoding="ISO-8859-1", parse_dates=['Date'])

In [21]:
new_zillow_df = zillow_df.rename(columns={'RegionName':'ZipCode','Zhvi':'zhvi'})

In [22]:
new_zillow_df['ZipName'] = zillow_df['City'].map(str) + ", " + zillow_df['State']

In [23]:
zillow_listings = new_zillow_df[['Date','ZipCode','zhvi']]

In [24]:
zillow_listings = zillow_listings.astype({"ZipCode": str})

In [25]:
cleaned_zillow_df = pd.merge(zillow_listings, zips_df, on="ZipCode")

In [26]:
print("There are " + str(len(cleaned_zillow_df)) + " zrows of zdata for zestimates in the zindex.")

There are 54 zrows of zdata for zestimates in the zindex.


In [27]:
cleaned_zillow_df.tail()

,Date,ZipCode,zhvi
49,2019-02-28,92359,333100
50,2019-02-28,92344,311200
51,2019-02-28,92339,240100
52,2019-02-28,92325,233000
53,2019-02-28,92241,230200


Known issue: Realtor.com uses the first day of the month, zillow uses last day.   
This need to be normalized by removing the day from the Date columns in the merge below.

In [28]:
new_merged_df = pd.merge(zip_listings, cleaned_zillow_df,  how='left', left_on=['Date','ZipCode'], right_on = ['Date','ZipCode'])
new_merged_df.head()

,Date,ZipCode,ZipName,Median Listing Price,Avg Listing Price,zhvi
0,2019-02-01,92241,"Desert Hot Springs, CA",249950.0,433049.0,NaN
1,2019-01-01,92241,"Desert Hot Springs, CA",252225.0,454677.0,NaN
2,2018-12-01,92241,"Desert Hot Springs, CA",254500.0,488984.0,NaN
3,2018-11-01,92241,"Desert Hot Springs, CA",275000.0,471347.0,NaN
4,2018-10-01,92241,"Desert Hot Springs, CA",289950.0,431134.0,NaN


In [29]:
len(new_merged_df)

3526

## Census Data by Zip Code

In [30]:
zip_stats = pd.read_csv(zip_code_stats, encoding="ISO-8859-1")

In [31]:
zip_stats.head(5)

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HC01,HC02,HC04,HC05,HC06,HC08,HC09
0,Id,Id2,Geography,Target Geo Id,Target Geo Id2,Geographical Area,Geographical Area,Population,Housing units,Area in square miles - Total area,Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Density per square mile of land area - Housing...
1,8500000US900,900,ZCTA3 900,8500000US900,900,ZCTA 900,ZCTA 900,2376389,857386,220.57,1.17,219.40,10831.3,3907.9
2,8500000US900,900,ZCTA3 900,8600000US90001,90001,ZCTA 900 - ZCTA 90001,ZCTA 90001,54481,13302,3.47,0.00,3.47,15688.0,3830.4
3,8500000US900,900,ZCTA3 900,8600000US90002,90002,ZCTA 900 - ZCTA 90002,ZCTA 90002,44584,11838,3.04,0.00,3.04,14677.4,3897.2
4,8500000US900,900,ZCTA3 900,8600000US90003,90003,ZCTA 900 - ZCTA 90003,ZCTA 90003,58187,16071,3.54,0.00,3.54,16440.6,4540.8


In [32]:
header = zip_stats.iloc[0]
new_zip_stats = zip_stats[1:]

In [33]:
new_zip_stats.head()

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HC01,HC02,HC04,HC05,HC06,HC08,HC09
1,8500000US900,900,ZCTA3 900,8500000US900,900,ZCTA 900,ZCTA 900,2376389,857386,220.57,1.17,219.40,10831.3,3907.9
2,8500000US900,900,ZCTA3 900,8600000US90001,90001,ZCTA 900 - ZCTA 90001,ZCTA 90001,54481,13302,3.47,0.00,3.47,15688.0,3830.4
3,8500000US900,900,ZCTA3 900,8600000US90002,90002,ZCTA 900 - ZCTA 90002,ZCTA 90002,44584,11838,3.04,0.00,3.04,14677.4,3897.2
4,8500000US900,900,ZCTA3 900,8600000US90003,90003,ZCTA 900 - ZCTA 90003,ZCTA 90003,58187,16071,3.54,0.00,3.54,16440.6,4540.8
5,8500000US900,900,ZCTA3 900,8600000US90004,90004,ZCTA 900 - ZCTA 90004,ZCTA 90004,67850,23922,3.06,0.00,3.06,22196.4,7825.8


In [34]:
new_zip_stats.columns = header
new_zip_stats.columns

Index(['Id', 'Id2', 'Geography', 'Target Geo Id', 'Target Geo Id2',
       'Geographical Area', 'Geographical Area', 'Population',
       'Housing  units', 'Area in square miles - Total area',
       'Area in square miles - Water area', 'Area in square miles - Land area',
       'Density per square mile of land area - Population',
       'Density per square mile of land area - Housing  units'],
      dtype='object', name=0)

In [35]:
zip_stats = new_zip_stats[['Target Geo Id2', 'Population', 'Housing  units', 'Area in square miles - Total area', 'Density per square mile of land area - Population', 'Density per square mile of land area - Housing  units']]

In [36]:
# Remove anything that does not have a 5-digit zip format (eg, "900" aggregation rows for 900xx)
final_zip_stats = zip_stats[zip_stats['Target Geo Id2'].str.match('\d{5}$')]

In [37]:
zip_stats_count = len(zip_stats)
final_zip_stats_count = len(final_zip_stats)
print("Zip Data: " + str(zip_stats_count) + " rows")
print("Filtered Zip Data: " + str(final_zip_stats_count) + "rows")
print("Removed " + str((zip_stats_count - final_zip_stats_count)) + " non-zip code rows" )

Zip Data: 1890 rows
Filtered Zip Data: 1752rows
Removed 138 non-zip code rows


In [38]:
second_final_df = final_zip_stats.rename(columns={'Target Geo Id2':'ZipCode'})
final_zip_stats = second_final_df

In [39]:
merged_datasets = pd.merge(new_merged_df, final_zip_stats, how="left", on="ZipCode")

In [40]:
#merged_datasets.info()
merged_datasets.head()

,Date,ZipCode,ZipName,Median Listing Price,Avg Listing Price,zhvi,Population,Housing units,Area in square miles - Total area,Density per square mile of land area - Population,Density per square mile of land area - Housing units
0,2019-02-01,92241,"Desert Hot Springs, CA",249950.0,433049.0,NaN,5512,6043,121.23,45.5,49.8
1,2019-01-01,92241,"Desert Hot Springs, CA",252225.0,454677.0,NaN,5512,6043,121.23,45.5,49.8
2,2018-12-01,92241,"Desert Hot Springs, CA",254500.0,488984.0,NaN,5512,6043,121.23,45.5,49.8
3,2018-11-01,92241,"Desert Hot Springs, CA",275000.0,471347.0,NaN,5512,6043,121.23,45.5,49.8
4,2018-10-01,92241,"Desert Hot Springs, CA",289950.0,431134.0,NaN,5512,6043,121.23,45.5,49.8


Notes:   
There will be NaNs. That's fine, as some zips have a zhvi, others have Median listing price.  
Still need to get the date alignment between Zillow and Realtor.com to match up, though.

Columns:
- Date: Obvious. 
- ZipCode: Five digit US zip code
- Median Listing Price: Median **listing** price from realtor.com
- Avg Listing Price: Mean **listing** price from realtor.com
- zhvi: Zillow Home Value Index, median **Zestimate** value from Zillow Research.
- Population: US Census, total population for zip code
- Housing Units: US Census, total housing units in zip code
- Area and Density: US Census